<a href="https://colab.research.google.com/github/fhfmendes04/HA_FJSSP/blob/main/HA_AG_TS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random


In [ ]:
class individuo():
  def __init__(self, job, operation, machine, regras_MS):
    self.OS = []
    self.MS = []
    self.fitness = 0
    self.regras_MS = regras_MS

    self.job = job
    self.operation = operation
    self.machine = machine

    self.OS_String()
    self.MS_String()

  def OS_String(self):
    lista = [12,12,12,12]
    for l in range(0,self.job * self.operation):
      flag = True
      while flag:
        numero = random.randint(0,3)
        if lista[numero]>0:
          lista[numero] -= 1
          self.OS.append(numero)
          flag=False


  def MS_String(self):
    for j in range(0, self.job):
      reg = self.regras_MS[j]
      for o in range(0, self.operation):
        self.MS.append(random.choice(reg[o]))







In [ ]:
class HA_AG_TS():
  def __init__(self, tamanho_populacao=400, geracao=200, prob_crossover=0.8, prob_mutacao=0.1):
    self.tamanho_populacao = tamanho_populacao
    self.populacao = []
    self.geracao = geracao
    self.prob_crossover = prob_crossover
    self.prob_mutacao = prob_mutacao
    self.lista_solucoes = []
    self.df = pd.DataFrame()
    self.job = 0
    self.operation = 0
    self.machine = 0
    self.regras_MS = []

    self.upload()
    self.regramento_MS()

  def upload(self):
      arquivo = "/content/drive/MyDrive/USP/Mestrado Sistema de Informação/SIN 5006 - Inteligência Computacional/Apresentação I/code/MFJS01.xlsx"
      self.df = pd.read_excel(arquivo, index_col=False)
      self.job = len(self.df['JOB'].unique())
      self.operation = len(self.df['OPERATION'].unique())
      self.machine = len(self.df['MACHINE'].unique())
      print("JOB: ", self.job)
      print("OPERATION: ", self.operation)
      print("MACHINE: ", self.machine)

      self.df.fillna(0,inplace=True)
      self.df["MACHINE.2"] = self.df["MACHINE.2"].astype(int)
      self.df['TIME.2'] = self.df["TIME.2"].astype(int)

  def regramento_MS(self):
    for j in range(0,self.job):
      regra_o = []
      dados = self.df[self.df["JOB"]==j].reset_index(drop=True)
      for o in range(0, self.operation):
        regra = []
        machine_1 = dados.iloc[o][3]
        machine_2 = dados.iloc[o][5]
        machine_3 = dados.iloc[o][7]

        if (machine_1 != 0):
          regra.append(machine_1)
        elif (machine_1 == 0):
          if (dados.iloc[0][4]!= 0):
            regra.append(machine_1)

        if(machine_2 != 0):
          regra.append(machine_2)
        elif (machine_2 == 0):
          if(dados.ilo[o][6] != 0):
            regra.append(machine_2)

        if(machine_3 !=0):
          regra.append(machine_3)
        elif (machine_3 == 0):
          if (dados.iloc[o][8] !=0):
            regra.append(machine_3)

        regra_o.append(regra)

      self.regras_MS.append(regra_o)

  def fitness(self,os_lista, ms_lista):
    maquinas = []
    controle_os = np.zeros(self.operatio)
    for m in range(0,self.machine):
      maquinas.apppend(np.zeros(5000))


    for o in os_lista:
      posicao = controle_os[o]
      controle_os[o] += 1
      self.df[self.dff["JOB"]==o]









In [ ]:
algoritmo.df.head()

,JOB,OPERATION,NUMBER_MACHINE,MACHINE,TIME,MACHINE.1,TIME.1,MACHINE.2,TIME.2
0,0,0,3,0,247,1,223,2,100
1,0,1,3,3,140,1,130,6,123
2,0,2,3,3,150,4,160,6,200
3,0,3,2,6,210,7,145,0,0
4,1,0,2,0,214,2,150,0,0


In [ ]:
algoritmo = HA_AG_TS()

JOB:  12
OPERATION:  4
MACHINE:  8


In [ ]:
ind = individuo(12, 4, 8, algoritmo.regras_MS)

In [ ]:
ind.MS

[2,
 6,
 3,
 6,
 0,
 2,
 6,
 4,
 0,
 6,
 3,
 7,
 1,
 2,
 3,
 4,
 2,
 6,
 5,
 7,
 3,
 4,
 4,
 3,
 1,
 4,
 4,
 4,
 2,
 4,
 5,
 7,
 1,
 2,
 4,
 7,
 3,
 5,
 4,
 7,
 2,
 2,
 6,
 5,
 1,
 2,
 6,
 4]